# When Cheryl Met Eve: A Birthday Story

Peter Norvig, May 2015

The "Cheryl's Birthday" logic puzzle  [made the rounds](https://www.google.com/webhp?#q=cheryl%27s+birthday),
and  I wrote [code](Cheryl.ipynb) that solves it. There I said that one reason for solving the problem with code rather than pencil and paper is that you can do more with code.  [Gabe Gaster](http://www.gabegaster.com/) proved me right when he [tweeted](https://twitter.com/gabegaster/status/593976413314777089/photo/1)  that he had extended my code to generate a new list of dates that satisfies the constraints of the puzzle:

     January 15, January 4,
     July 13, July 24, July 30,
     March 13, March 24,
     May 11, May 17, May 30

In this notebook, I verify Gabe's result, and find some other variations on the puzzle.

First, let's recap  [the puzzle](https://en.wikipedia.org/wiki/Cheryl%27s_Birthday):

> Albert and Bernard just became friends with Cheryl, and they want to know when her birthday is. Cheryl gives them a list of 10 possible dates:

>     May       15 16       19
>     June            17 18	
>     July   14    16			
>     August 14 15    17		

> Cheryl then tells Albert and Bernard separately the month and the day of her birthday respectively.

>**Albert**: I don't know when Cheryl's birthday is, but I know that Bernard does not know too.

>**Bernard**: At first I don't know when Cheryl's birthday is, but I know now.

>**Albert**: Then I also know when Cheryl's birthday is.

>So when is Cheryl's birthday?"

# Code for Original Cheryl's Birthday Puzzle

This is a slight modification of my [previous code](Cheryl.ipynb), and I'll give a slight modification of the explanation. The puzzle concerns these concepts:

- **Possible dates** that might be Cheryl's birthday.
- **Knowing** which dates are still possible; knowing for sure when only one is possible.
- **Telling** Albert and Bernard specific facts about the birthday.
- **Statements** about knowledge.
- **Hearing** the statements about knowledge.

I implement them as follows:
- `DATES` is a set of all possible dates (each date is a string); we also consider subsets of `DATES`.
- `know(possible_dates)` is a function that returns `True` when there is only one possible date.
- `tell(part)` is a function that returns the set of possible dates after Cheryl tells a part (month or day).
- *`statement`*`(date)` returns true for dates that satisfy the statement.
- `hear(possible_dates, statement,...)` returns a subset of possible_dates that are still possible after hearing the statements.

For example, `albert1` is Albert's first statement, which says in part that he knows that Bernard
does not yet know Cheryl's birthday. So `albert1('May 19')` should be `False`, because if May 19 were Cheryl's birthday, then she would have told Bernard `'19'`, he would know that `'May 19'` is the only possibility.

`tell('17')` returns `{'June 17', 'August 17'}`, the set of possible dates after Bernard is told `'17'`. 

 `know(tell('17'))` is `False` because if Bernard was told `'17'` he would not know Cheryl's birthday for sure; there would be two possibilities. But `know(tell('19'))` is `True` because there would be only one possibility.


In [1]:
from operator import *
DATAS={}
BeliefSet = set

def tell(part, op=contains) -> BeliefSet:
    "tells a part of DATAS to someone; acording justyfy op return a set of possible datas."
    return {data for data in DATAS if op(data,part)}

def know(beliefs) -> bool:
    "A person `knows` the answer if their belief set has only one possibility."
    return len(beliefs) == 1

def hear(items:set, *predicates) -> BeliefSet:
    "Return the subset of items that hear all the predicates."
    return {item for item in items
            if all(pred(item) for pred in predicates)}

def using(datas):
    "init global variable DATAS."
    global DATAS # This is necessary because `tell` looks at `DATES`
    DATAS = datas
    return datas

In [13]:
# Albert and Bernard just became friends with Cheryl, and they want to know when her birthday is. 
# Cheryl gives them a set of 10 possible dates:

cheryl_dates =  {
     'May 15',    'May 16',    'May 19',
    'June 17',   'June 18',
    'July 14',   'July 16',
  'August 14', 'August 15', 'August 17'}

def month(date): return date.split()[0]

def day(date):   return date.split()[1]

# Cheryl then tells Albert and Bernard separately 
# the month and the day of the birthday respectively.

# Albert and Bernard make three statements:

def albert1(date):
    "Albert: I don't know when Cheryl's birthday is, but I know that Bernard does not know too."
    after_being_told = tell(month(date))
    return (not know(after_being_told) 
            and all(not know(tell(day(d)))
                    for d in after_being_told))

def bernard1(date):
    "Bernard: At first I don't know when Cheryl's birthday is, but I know now."
    at_first = tell(day(date))
    return (not know(at_first)
            and know(hear(at_first, albert1)))

def albert2(date):
    "Albert: Then I also know when Cheryl's birthday is."
    return know(hear(tell(month(date)), bernard1))
    
# So when is Cheryl's birthday?

def cheryls_birthday(dates):
    "Return a list of the possible dates after hearing the three statements."
    return hear(using(dates), albert1, bernard1, albert2)

using(cheryl_dates)

# Some tests

assert month('May 19') == 'May'
assert day('May 19') == '19'
assert albert1('May 19') == False
assert albert1('July 14') == True
assert know(tell('17')) == False
assert know(tell('19')) == True

In [37]:
using(cheryl_dates)

{'August 14',
 'August 15',
 'August 17',
 'July 14',
 'July 16',
 'June 17',
 'June 18',
 'May 15',
 'May 16',
 'May 19'}

In [14]:
DATAS

{'August 14',
 'August 15',
 'August 17',
 'July 14',
 'July 16',
 'June 17',
 'June 18',
 'May 15',
 'May 16',
 'May 19'}

In [38]:
tell('May')

{'May 15', 'May 16', 'May 19'}

In [39]:
tell('')

{'August 14',
 'August 15',
 'August 17',
 'July 14',
 'July 16',
 'June 17',
 'June 18',
 'May 15',
 'May 16',
 'May 19'}

In [15]:
cheryls_birthday(cheryl_dates)

{'July 16'}

In [16]:
hear(cheryl_dates, albert1)

{'August 14', 'August 15', 'August 17', 'July 14', 'July 16'}

In [17]:
hear(cheryl_dates, albert1, bernard1)

{'August 15', 'August 17', 'July 16'}

In [18]:
hear(cheryl_dates, albert1, bernard1, albert2)

{'July 16'}

# Verifying Gabe's Version

Gabe tweeted these ten dates:

In [19]:
gabe_dates = [
  'January 15', 'January 4',
  'July 13',    'July 24',   'July 30',
  'March 13',   'March 24',
  'May 11',     'May 17',    'May 30']

We can verify that they do indeed make the puzzle work, giving a single known birthdate:

In [20]:
cheryls_birthday(gabe_dates)

{'July 30'}

# Creating Our Own Versions

If Gabe can do it, we can do it!  Our strategy will be to repeatedly pick a random sample of dates, and check if they solve the puzzle. We'll limit ourselves to a subset of dates (not all 366) to make it more likely that a random selection will have multiple dates with the same month and day (otherwise Albert and Bernard would know right away):

Now we need to cycle through random samples of these possible dates until we hit one that works.  I anticipate wanting to solve other puzzles besides the original `cheryls_birthday`, so I'll make the `puzzle` be a parameter of the function `pick_dates`. Note that `pick_dates` returns two things: the one date that is the solution (the birthday), and the `k` (default 10) dates that form the puzzle.

In [64]:
import random

def pick_dates(puzzle=cheryls_birthday, k=10):
    "Pick a set of dates for which the puzzle has a unique solution."
    while True:
        dates = set(random.sample(some_dates, k))
        solutions = puzzle(dates)
        if know(solutions):
            return solutions.pop(), dates

In [23]:
pick_dates()

('April 26',
 {'April 18',
  'April 26',
  'April 28',
  'July 17',
  'July 26',
  'July 29',
  'June 26',
  'March 18',
  'March 28',
  'May 28'})

Great! We can make a new puzzle, just like Gabe.  But how often do we get a unique solution to the puzzle (that is, the puzzle returns a set of size 1)?  How often do we get a solution where Albert and Bernard know, but we the puzzle solver doesn't (that is, a set of size greater than 1)?  How often is there no solution (size 0)? Let's make a Counter of the number of times each length-of-solution occurs:

In [65]:
from collections import Counter

def counter(puzzle=cheryls_birthday, N=10000, k=10):
    "Try N random samples and count how often each possible length-of-solution appears."
    return Counter(len(puzzle(set(random.sample(some_dates, k))))
                   for _ in range(N))

In [66]:
counter(cheryls_birthday)

Counter({1: 593, 0: 8709, 2: 696, 3: 2})

This says that about 6% of the time we get a unique solution (a set of `len` 1). With similar frequency we get an ambiguous solution (with 2 or more possible birth dates).  And most of the time, the sample of dates leads to no solution.

What happens if Cheryl changes the number of possible dates?

In [67]:
counter(cheryls_birthday, k=6)

Counter({0: 9952, 2: 28, 1: 20})

In [68]:
counter(cheryls_birthday, k=12)

Counter({0: 7551, 2: 1065, 1: 1345, 3: 39})

It is really hard (but not impossible) to find a set of 6 dates that work for the puzzle, and much easier to find a solution with 12 dates.

# A New Puzzle: All About Eve

Now let's see if we can create a more complicated puzzle. We'll introduce a new character, Eve, give her a statement, and alter the rest of the puzzle slightly:

> *Albert and Bernard just became friends with Cheryl, and they want to know when her birthday is. Cheryl wrote down a list of 10 possible dates for all to see.*
    
> *Cheryl then writes down the month and shows it just to Albert, and also writes down the day and shows it just to Bernard.*

> **Albert**: *I don't know when Cheryl's birthday is, but I know that Bernard does not know either.*

> **Bernard**: *At first I didn't know when Cheryl's birthday is, but I know now.*
    
> **Albert**: *Then I also know when Cheryl's birthday is.*

> **Eve**: *Hi, my name is Eve and I'm an evesdropper. It's what I do! I peeked and saw the first letter of the month and the first digit of the day. When I peeked, I didn't know Cheryl's birthday, but after listening to Albert and Bernard I do.  And it's a good thing I peeked, because otherwise I couldn't have
figured it out.*
    
> *So when is Cheryl's birthday?*

We can easily code this up:

In [69]:
def cheryls_birthday_with_eve(dates):
    "Return a set of the dates for which Albert, Bernard, and Eve's statements are true."
    return hear(using(dates), albert1, bernard1, albert2, eve1)

def eve1(date):
    """Eve: I peeked and saw the first letter of the month and the first digit of the day. 
    When I peeked, I didn't know Cheryl's birthday, but after listening to Albert and Bernard 
    I do. And it's a good thing I peeked, because otherwise I couldn't have figured it out."""
    at_first = tell(first(day(date)),hasdayfirst) & tell(first(month(date)),hasmonthfirst)
    otherwise = tell('')
    return (not know(at_first) and
                know(hear(at_first, albert1, bernard1, albert2)) and
            not know(hear(otherwise, albert1, bernard1, albert2)))

def first(seq): return seq[0]

def hasdayfirst(data,firstword): return day(data)[0]==firstword
def hasmonthfirst(data,firstword): return month(data)[0]==firstword

*Note*: I admit I "cheated" a bit here.  Remember that the function `tell`  tests for `(part in date)`.  For that to work for Eve, we have to make sure that the first letter is distinct from any other character in the date (it is&mdash;because only the first letter is uppercase) and that the first digit is distinct from any other character (it is&mdash;because in `some_dates` I carefully made sure that the first digit is always 1 or 2, and the second digit is never 1 or 2). Also note that `tell('')` denotes the hypothetical situation where Cheryl "told" Eve nothing.

I have no idea if it is possible to find a set of dates that works for this puzzle. But I can try:

In [70]:
aa=pick_dates(puzzle=cheryls_birthday_with_eve)

In [71]:
aa

('March 18',
 {'April 15',
  'April 28',
  'July 28',
  'June 15',
  'June 27',
  'March 15',
  'March 18',
  'March 27',
  'May 18',
  'May 29'})

In [72]:
cheryls_birthday_with_eve(aa[1])

{'March 18'}

In [73]:
hear(using(aa[1]), albert1, bernard1, albert2)

{'April 28', 'March 18'}

That was easy.  How often is a random sample of dates a solution to this puzzle?

In [74]:
DATAS

{'April 15',
 'April 28',
 'July 28',
 'June 15',
 'June 27',
 'March 15',
 'March 18',
 'March 27',
 'May 18',
 'May 29'}

In [75]:
counter(cheryls_birthday_with_eve)

Counter({0: 9390, 2: 306, 1: 304})

counter(cheryls_birthday_with_eve)  
Counter({0: 9492, 1: 258, 2: 250})

About half as often as for the original puzzle.

# An Even More Complex Puzzle

Let's make the puzzle even more complicated by making Albert wait one more time before he finally knows:

> *Albert and Bernard just became friends with Cheryl, and they want to know when her birtxhday is. Cheryl wrote down a list of 10 possible dates for all to see.*
    
> *Cheryl then writes down the month and shows it just to Albert, and also writes down the day and shows it just to Bernard.*
    
> **Albert**: *I don't know when Cheryl's birthday is, but I know that Bernard does not know either.*
    
> **Bernard**: *At first I didn't know when Cheryl's birthday is, but I know now.*
    
> **Albert**: *I still don't know.*

> **Eve**: *Hi, my name is Eve and I'm an evesdropper. It's what I do! I peeked and saw the first letter of the month and the first digit of the day. When I peeked, I didn't know Cheryl's birthday, but after listening to Albert and Bernard I do.  And it's a good thing I peeked, because otherwise I couldn't have figured it out.*

> **Albert**: *OK, now I know.*
    
> *So when is Cheryl's birthday?*

Let's be careful in coding this up; Albert's second statement is different; he has a new third statement; and Eve's statement uses the same words, but it now implicitly refers to a different statement by Albert. We'll use the names `albert2c`,  `eve1c`, and `albert3c` (`c` for "complex") to represent the new statements:

In [78]:
def cheryls_birthday_complex(dates):
    "Return a set of the dates for which Albert, Bernard, and Eve's statements are true."
    return hear(using(dates), albert1, bernard1, albert2c, eve1c, albert3c)

def albert2c(date):
    "Albert: I still don't know."
    return not know(hear(tell(month(date)), bernard1))

def eve1c(date):
    """Eve: I peeked and saw the first letter of the month and the first digit of the day. 
    When I peeked, I didn't know Cheryl's birthday, but after listening to Albert and Bernard 
    I do. And it's a good thing I peeked, because otherwise I couldn't have figured it out."""
    at_first = tell(first(day(date)),hasdayfirst) & tell(first(month(date)),hasmonthfirst)
    otherwise = tell('')
    return (not know(at_first)
            and know(hear(at_first, albert1, bernard1, albert2c)) and
            not know(hear(otherwise, albert1, bernard1, albert2c)))

def albert3c(date):
    "Albert: OK, now I know."
    return know(hear(tell(month(date)), eve1c))

def hasdayfirst(data,firstword): return day(data)[0]==firstword
def hasmonthfirst(data,firstword): return month(data)[0]==firstword

Again, I don't know if it is possible to find dates that works with this story, but I can try:

In [79]:
pick_dates(puzzle=cheryls_birthday_complex)

('May 26',
 {'July 18',
  'June 15',
  'June 19',
  'June 26',
  'March 18',
  'March 19',
  'March 25',
  'May 18',
  'May 19',
  'May 26'})

It worked!  Were we just lucky, or are there many sets of dates that work?

In [57]:
counter(cheryls_birthday_complex)

Counter({0: 9136, 1: 862, 2: 2})

counter(cheryls_birthday_complex)  
Counter({0: 9047, 1: 951, 2: 2})

Interesting. It was actually easier to find dates that work for this story than for either of the other stories.

## Analyzing a Solution to the Complex Puzzle

Now we will go through a solution step-by-step.  We'll use a set of dates selected in a previous run:

In [81]:
DATES = {
  'April 28',
  'July 27',
  'June 19',
  'June 16',
  'July 15',
  'April 15',
  'June 29',
  'July 16',
  'May 24',
  'May 27'}
  
using(DATES)

{'April 15',
 'April 28',
 'July 15',
 'July 16',
 'July 27',
 'June 16',
 'June 19',
 'June 29',
 'May 24',
 'May 27'}

Let's find the solution:

In [82]:
cheryls_birthday_complex(DATES)

{'July 27'}

Now the first step is that Albert was told "July":

In [83]:
tell('July')

{'July 15', 'July 16', 'July 27'}

And no matter which of these three dates is the actual birthday, Albert knows that Bernard would not know the birthday, because each of the days (15, 16, 27) appears twice in the list of possible dates.

In [84]:
all(not know(tell(day(d)))
    for d in tell('July'))

True

Next, Bernard is told the day:

In [85]:
tell('27')

{'July 27', 'May 27'}

There are two dates with a 27, so Bernard did not know then. But only one of these dates is still consistent after hearing Albert's statement:

In [86]:
hear(tell('27'), albert1)

{'July 27'}

So after Albert's statement, Bernard knows. Poor Albert still doesn't know (after being told `'July'` and hearing Bernard's statement):

In [87]:
hear(tell('July'), bernard1)

{'July 15', 'July 16', 'July 27'}

Then along comes Eve. She evesdrops the "J" and the "2":

In [88]:
tell('J',hasmonthfirst) & tell('2',hasdayfirst)

{'July 27', 'June 29'}

Two dates, so Eve doesn't know yet. But only one of the dates works after hearing the three statements made by Albert and Bernard:

In [89]:
hear(tell('J') & tell('2'), albert1, bernard1, albert2c)

{'July 27'}

But Eve wouldn't have known if she had been told nothing:

In [90]:
hear(tell(''), albert1, bernard1, albert2c)

{'July 15', 'July 16', 'July 27'}

What about Albert?  After hearing Eve's statement he finally knows:

In [91]:
hear(tell('July'), eve1c)

{'July 27'}

# Three Children

Here's another puzzle:

> *A parent has the following conversation with a friend:*

> **Parent:** the product of my three childrens' ages is 36.

> **Friend**: I don't know their ages.

> **Parent**: The sum of their ages is the same as the number of people in this room.

> **Friend**: I still don't know their ages.

> **Parent**: The oldest one likes bananas.

> **Friend**: Now I know their ages.

Let's follow the same methodology to solve this puzzle. Except this time, we're not dealing with sets of possible dates, we're dealing with set of possible *states* of the world. We'll define a state as a tuple of 4 numbers: the ages of the three children (in increasing order), and the number of people in the room. (We'll limit the children's ages to be below 30 and the number of people in the room to be below 90.)

In [58]:
N      = 30
states = {(a, b, c, n) 
          for a in range(1, N)
          for b in range(a, N)
          for c in range(b, N) if a * b * c == 36
          for n in range(2, 90)}

def ages(state): return state[:-1]
def room(state): return state[-1]

def parent1(state): 
    "The product of my three childrens' ages is 36."
    a, b, c = ages(state)
    return a * b * c == 36

def friend1(state): 
    "I don't know their ages."
    return not know({ages(s) for s in hear(states, parent1)})

def parent2(state):
    "The sum of their ages is the same as the number of people in this room."
    return sum(ages(state)) == room(state)

def friend2(state, possibilities=hear(states, parent1, friend1, parent2)): 
    "I still don't know their ages."
    # Given there are room(state) people in the room, I still don't know the ages.
    return not know({ages(s) for s in possibilities if room(s) == room(state)})

def parent3(state):
    "The oldest one likes bananas."
    # I.e., there is an oldest one (and not twins of the same age)
    a, b, c = ages(state)
    return c > b

def friend3(state, possibilities=hear(states, parent1, friend1, parent2, friend2, parent3)): 
    "Now I know their ages."
    return know({ages(s) for s in possibilities})

def child_age_puzzle(states):
    return hear(states, parent1, friend1, parent2, friend2, parent3, friend3)

child_age_puzzle(states)

{(2, 2, 9, 13)}

The tricky part of this puzzle comes after the `parent2` statement:

In [33]:
hear(states, parent1, friend1, parent2)

{(1, 2, 18, 21),
 (1, 3, 12, 16),
 (1, 4, 9, 14),
 (1, 6, 6, 13),
 (2, 2, 9, 13),
 (2, 3, 6, 11),
 (3, 3, 4, 10)}

We see that out of these 7 possibilities, if the number of people in the room (the last number in each tuple) 
were anything other than 13, then the friend (who can observe the number of people in the room) would know the ages. Since the `friend2` statement professes continued ignorance, it must be that the number of people in the room is 13. Then the `parent3` statement makes it clear that there can't be 6-year-old twins as the oldest children; it must be 2-year-old twins with an oldest age 9.

# Sum and Product Puzzle

X and Y are two different whole numbers greater than 1. Their sum is no greater than 100, and Y is greater than X. S and P are two mathematicians (and consequently perfect logicians); S knows the sum X+Y and P knows the product X*Y. Both S and P know all the information in this paragraph.

The following conversation occurs:

- S says "P does not know X and Y."
- P says "Now I know X and Y."
- S says "Now I also know X and Y!"

What are X and Y?


In [120]:
%%time

N      = 100
states = {(x, y, x+y, x*y) 
          for x in range(2, N)
          for y in range(x+1, N) if x+y <= 100}

def sum_of(state): return state[2]
def product_of(state): return state[3]

def equal_sum(state,s): return sum_of(state) == s
def equal_product(state,p): return product_of(state) == p

def s1(state): 
    "S says 'P does not know X and Y.'"
    after_being_told = tell(sum_of(state),equal_sum)
    return all(not know(tell(product_of(d),equal_product)) for d in after_being_told)

def p1(state): 
    "P says 'Now I know X and Y.'"
    after_being_told = tell(product_of(state),equal_product)
    return know(hear(after_being_told, s1))

def s2(state):
    "S says 'Now I also know X and Y!'"
    after_being_told = tell(sum_of(state),equal_sum)
    return know(hear(after_being_told,p1))

def sum_product_puzzle(states):
    return hear(using(states), s1, p1, s2)

sum_product_puzzle(states)

Wall time: 15 ms


set()

In [108]:
%%time
sum_product_puzzle(states)

Wall time: 12 ms


set()

In [51]:
%%time

N      = 100
states = {(x, y, x+y, x*y) 
          for x in range(2, N)
          for y in range(x+1, N) if x+y <= 100}

def sum_of(state): return state[2]
def product_of(state): return state[3]

def equal_sum(state,s): return sum_of(state) == s
def equal_product(state,p): return product_of(state) == p

def s1(state): 
    "S says 'P does not know X and Y.'"
    after_being_told = tell(sum_of(state),equal_sum)
    return all(not know(tell(product_of(d),equal_product)) for d in after_being_told)

def p1(state): 
    "P says 'Now I know X and Y.'"
    after_being_told = tell(product_of(state),equal_product)
    return know(hear(after_being_told, s1))

def s2(state,possibilities=hear(states, s1, p1)):
    "S says 'Now I also know X and Y!'"
    return know({x for x in possibilities if sum_of(state) == sum_of(x)})

def sum_product_puzzle(states):
    ls =hear(using(states), s1)
    ls =hear(ls, p1)
    ls =hear(ls, s2)
    return ls

sum_product_puzzle(states)

Wall time: 34 ms


set()

In [ ]:
from operator import *
DATAS={}
BeliefSet = set

def tell(part, op=contains) -> BeliefSet:
    "tells a part of DATAS to someone; acording justyfy op return a set of possible datas."
    return {data for data in DATAS if op(data,part)}

def know(beliefs) -> bool:
    "A person `knows` the answer if their belief set has only one possibility."
    return len(beliefs) == 1

def hear(items:set, *predicates) -> BeliefSet:
    "Return the subset of items that hear all the predicates."
    return {item for item in items
            if all(pred(item) for pred in predicates)}

def using(datas):
    "init global variable DATAS."
    global DATAS # This is necessary because `tell` looks at `DATES`
    DATAS = datas
    return datas

In [9]:
a={(1,2),(3,4)}
list(a)

[(1, 2), (3, 4)]

In [43]:
from operator import *
DATAS = set()
BeliefSet = set

def tell(part,op=contains,items=DATAS) -> BeliefSet:
    "tells a part of DATAS to someone; acording justyfy op return a set of possible datas."
    return set(filter(lambda x:op(x,part),list(items)))

def know(beliefs) -> bool:
    "A person `knows` the answer if their belief set has only one possibility."
    return len(beliefs) == 1

def hear(items:set, *predicates) -> BeliefSet:
    "Return the subset of items that hear all the predicates."
    for pred in predicates:
        items = set(filter(pred,list(items)))
    return items

def using(datas):
    "init global variable DATAS."
    global DATAS # This is necessary because `tell` looks at `DATES`
    DATAS = datas
    return datas

In [24]:
f = filter(lambda x:equal_sum(x,100),list(DATAS))

In [44]:
print(tell(100,equal_sum))

set()


In [48]:
def f(part,op=contains,items=DATAS):
  print(locals())#在函数内获取
  print(items)

In [52]:
f(100)

{'part': 100, 'op': <built-in function contains>, 'items': set()}
set()


In [99]:
def sum_product_puzzle(states):
%%time
    ls =hear(using(states), s1)
    ls =hear(ls, p1)
    ls =hear(ls, s2)
    return ls

sum_product_puzzle(states)

Wall time: 21.7 s


{(4, 13, 17, 52)}

In [110]:
%time ls1 =hear(using(states), s1)
%time ls2 =hear(ls1, p1)
%time ls3 =hear(ls2, s2)

Wall time: 5 ms
Wall time: 8 ms
Wall time: 0 ns


In [122]:
N      = 100
states = {(x, y, x+y, x*y) 
          for x in range(2, N)
          for y in range(x+1, N) if x+y <= 100}

In [11]:
tell(99,equal_sum)

set()

In [129]:
using(ls2)
tell(sum_of((4, 13, 17, 52)),equal_sum)

set()

In [71]:
len(ls1)

86

In [72]:
len(ls2)

86

In [33]:
from collections import Counter

N      = 100
states1 = {(x, y) 
          for x in range(2, N)
          for y in range(x+1, N) if x+y <= 100}
s_c=Counter((sum_of(s) for s in states1))
states2 = set(filter(lambda x:s_c[sum_of(x)] != 1,states1))
p_c=Counter((product_of(s) for s in states2))
states = set(filter(lambda x:p_c[product_of(x)] != 1,states2))


def sum_of(state): return state[0]+state[1]
def product_of(state): return state[0]*state[1]


In [34]:
len(states1),len(states2),len(states)

(2352, 2350, 1747)

In [ ]:
hear(using(states), s1)

In [82]:
%time
from collections import Counter
 
def decompose_sum(s):
    return [(a,s-a) for a in range(2,int(s/2+1))]
 
# Generate all possible pairs
all_pairs = set((a,b) for a in range(2,100) for b in range(a+1,100) if a+b<100)
 
# Fact 1 --> Select pairs for which all sum decompositions have non-unique product
product_counts = Counter(c*d for c,d in all_pairs)
unique_products = set((a,b) for a,b in all_pairs if product_counts[a*b]==1)
s_pairs = [(a,b) for a,b in all_pairs if
    all((x,y) not in unique_products for (x,y) in decompose_sum(a+b))]
 
# Fact 2 --> Select pairs for which the product is unique
product_counts = Counter(c*d for c,d in s_pairs)
p_pairs = [(a,b) for a,b in s_pairs if product_counts[a*b]==1]
 
# Fact 3 --> Select pairs for which the sum is unique
sum_counts = Counter(c+d for c,d in p_pairs)
final_pairs = [(a,b) for a,b in p_pairs if sum_counts[a+b]==1]
 
print(final_pairs)

Wall time: 0 ns
[(4, 13)]
